In [ ]:
#%pip install -q langchain langchain-openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [4]:
model  = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

#### 1. CommaSeparatedListOutputParser

In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# OpenAI 모델 설정
model  = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)


# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print("AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")


AI 관련 기술 목록:
['Machine Learning', 'Deep Learning', 'Natural Language Processing', 'Computer Vision', 'Robotics']
 'data/ai_technologies.csv' 파일로 저장 완료!


#### 2. JsonOutputParser

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# OpenAI 모델 설정
#model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


{
    "missions": [
        {
            "mission_name": "뉴호라이즌스(New Horizons)",
            "goal": "명왕성 및 카이퍼 벨트 탐사",
            "agency": "NASA"
        },
        {
            "mission_name": "로제타(Rosetta)",
            "goal": "혜성 67P/추리움 탐사",
            "agency": "ESA"
        },
        {
            "mission_name": "파커 태양 탐사선(Parker Solar Probe)",
            "goal": "태양의 코로나 및 태양풍 탐사",
            "agency": "NASA"
        }
    ]
}


#### 3. PandasDataFrameOutputParser

In [9]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# Titanic 데이터셋 로드
df = pd.read_csv('data/titanic/train.csv')

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
#print("Format Instructions:\n", format_instructions)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(list(df.columns))
    },
)

# 체인 생성
chain = prompt | model | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

        # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")



Name 컬럼 출력
오류 발생: Request 'The command for the user's query is:
```
column:Name
```' is not correctly formatted.                     Please refer to the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [15]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
#model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# 응답 스키마 정의
response_schemas = [
    ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are an AI assistant that generates tabular data. 
    You must return the data in JSON format that follows this schema:
    
    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 생성 (프롬프트 → 모델 → OutputParser)
chain = prompt | model | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})
        print(json_response)
        
        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n🔹 Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

In [16]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    "Create a dataset of the average apartment sale prices in Seoul for the first half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
)
df_seoul_housing.head()


2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
{'data': [{'District': 'Gangnam-gu', 'Average Price': 1500000000, 'Number of Transactions': 1200, 'Year-over-Year Change': 10.2}, {'District': 'Gangdong-gu', 'Average Price': 1200000000, 'Number of Transactions': 900, 'Year-over-Year Change': 8.5}, {'District': 'Gwangjin-gu', 'Average Price': 1800000000, 'Number of Transactions': 1500, 'Year-over-Year Change': 12.1}, {'District': 'Dobong-gu', 'Average Price': 900000000, 'Number of Transactions': 600, 'Year-over-Year Change': 6.3}, {'District': 'Dongdaemun-gu', 'Average Price': 1000000000, 'Number of Transactions': 800, 'Year-over-Year Change': 7.1}, {'District': 'Dongjak-gu', 'Average Price': 1100000000, 'Number of Transactions': 700, 'Year-over-Year Change': 8.2}, {'District': 'Eunpyeong-gu', 'Average Price': 1300000000, 'Number of Transactions': 1000, 'Year-over-Year Change': 9.5}, {'District': 'Geumcheon-gu', 'Average Price': 800000000, 'Number of Transactions': 500, 'Year-over-Year Change': 5.6}, {'

,District,Average Price,Number of Transactions,Year-over-Year Change
0,Gangnam-gu,1500000000,1200,10.2
1,Gangdong-gu,1200000000,900,8.5
2,Gwangjin-gu,1800000000,1500,12.1
3,Dobong-gu,900000000,600,6.3
4,Dongdaemun-gu,1000000000,800,7.1


In [17]:
print('2024년 서울 지하철역별 유동 인구 데이터')
# [예제 2] 2024년 서울 지하철역별 유동 인구 데이터
df_seoul_subway = generate_dataframe(
    "Generate a dataset of the top 10 busiest subway stations in Seoul in 2024 with columns: Station Name, Line Number, Daily Passenger Volume, and Weekday vs Weekend Ratio."
)
if df_seoul_subway is not None:
    df_seoul_subway.head()


2024년 서울 지하철역별 유동 인구 데이터
{'data': [{'Station Name': 'Gwanghwamun Station', 'Line Number': 'Line 5', 'Daily Passenger Volume': '145,621', 'Weekday vs Weekend Ratio': '1.23'}, {'Station Name': 'Gangnam Station', 'Line Number': 'Line 2', 'Daily Passenger Volume': '134,118', 'Weekday vs Weekend Ratio': '1.17'}, {'Station Name': 'Hongdae Station', 'Line Number': 'Line 2', 'Daily Passenger Volume': '126,479', 'Weekday vs Weekend Ratio': '1.21'}, {'Station Name': 'Yongsan Station', 'Line Number': 'Line 1, Line 4', 'Daily Passenger Volume': '123,456', 'Weekday vs Weekend Ratio': '1.19'}, {'Station Name': 'Itaewon Station', 'Line Number': 'Line 4', 'Daily Passenger Volume': '114,285', 'Weekday vs Weekend Ratio': '1.15'}, {'Station Name': 'Apgujeong Station', 'Line Number': 'Line 2', 'Daily Passenger Volume': '109,834', 'Weekday vs Weekend Ratio': '1.13'}, {'Station Name': 'Dongdaemun History & Culture Park Station', 'Line Number': 'Line 1, Line 4, Line 5', 'Daily Passenger Volume': '107,483', '

In [18]:
df_seoul_subway.head()

,Station Name,Line Number,Daily Passenger Volume,Weekday vs Weekend Ratio
0,Gwanghwamun Station,Line 5,"145,621",1.23
1,Gangnam Station,Line 2,"134,118",1.17
2,Hongdae Station,Line 2,"126,479",1.21
3,Yongsan Station,"Line 1, Line 4","123,456",1.19
4,Itaewon Station,Line 4,"114,285",1.15


In [19]:
print('한국 5대 편의점 브랜드별 2024년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    "Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
)
df_korean_convenience_stores.head()

한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
{'data': [{'Brand Name': 'GS25', 'Number of Stores': 18, 'Total Revenue': 11.5, 'Market Share': 34.6}, {'Brand Name': 'CU', 'Number of Stores': 15, 'Total Revenue': 9.8, 'Market Share': 29.4}, {'Brand Name': '7-Eleven', 'Number of Stores': 12, 'Total Revenue': 7.2, 'Market Share': 21.6}, {'Brand Name': 'E-Mart24', 'Number of Stores': 8, 'Total Revenue': 4.5, 'Market Share': 13.5}, {'Brand Name': 'Homeplus', 'Number of Stores': 5, 'Total Revenue': 3.2, 'Market Share': 9.6}]}

🔹 Generated DataFrame:



,Brand Name,Number of Stores,Total Revenue,Market Share
0,GS25,18,11.5,34.6
1,CU,15,9.8,29.4
2,7-Eleven,12,7.2,21.6
3,E-Mart24,8,4.5,13.5
4,Homeplus,5,3.2,9.6
